# 🎬 UVG MAX - Premium Video Generator

**$300 Fiverr Quality at $0 Cost**

Features:
- Fish-Speech S1 TTS (50+ emotion markers)
- CLIP-scored footage selection
- Emotion-driven VFX/SFX/Music
- Kinetic captions
- Auto thumbnails

**API Keys are PRE-CONFIGURED!** Just run all cells.

---

In [ ]:
# Cell 1: Clone & Install Dependencies
import os, sys

# Clear proxy settings
for var in ['http_proxy', 'https_proxy', 'HTTP_PROXY', 'HTTPS_PROXY']:
    os.environ.pop(var, None)

os.chdir('/content')
%rm -rf uvg-max-engine
!git clone https://github.com/shya-bubu/uvg-max-engine.git
os.chdir('uvg-max-engine')
sys.path.insert(0, '/content/uvg-max-engine')

# Core dependencies
%pip install -q python-dotenv requests tqdm torch
%pip install -q opencv-python Pillow numpy
%pip install -q librosa pyloudnorm moviepy scikit-learn
%pip install -q gradio

print('\n✅ Dependencies installed!')

In [ ]:
# Cell 2: API Keys (PRE-CONFIGURED - Just run!)
import os

# Clear proxy
for var in ['http_proxy', 'https_proxy', 'HTTP_PROXY', 'HTTPS_PROXY']:
    os.environ.pop(var, None)

# ============================================
# 🔐 ALL API KEYS PRE-CONFIGURED
# ============================================

# Video Providers
os.environ['PEXELS_KEY'] = '7QAyreSxu44EAJYIRfYXstHpvj1tof7v3Baj8tv8fvO4DV6l16I2FzlY'
os.environ['PIXABAY_KEY'] = '53451868-98a1100fbad21a5918d9610dd'
os.environ['UNSPLASH_ACCESS_KEY'] = 'tuL22IW04plnvq3HWbnLqY-rcDBDfMW_Aevf_yPHLOA'
os.environ['UNSPLASH_SECRET_KEY'] = '-zJrdrCCQJMw68jwvYo_uL-J44r8PATaen7y42FegdE'
os.environ['COVERR_KEY'] = '51d0dd8ca31b8dacad0fff6ce380d22c'
os.environ['FREEPIK_KEY'] = 'FPSX1b080f2c2f9d97c51f7fea57e58d5936'

# Audio Providers
os.environ['FREESOUND_KEY'] = 'RHEDFl5Y38hA0CLxwbivIZ2uExQWUiI56r41Fy40'

# Archive.org
os.environ['ARCHIVE_S3_ACCESS_KEY'] = 'akX7CJkcv90RQ2A9'
os.environ['ARCHIVE_S3_SECRET_KEY'] = 'utt6InZ5gjnNWmbR'

# UVG Settings
os.environ['UVG_MOCK_MODE'] = 'false'

# ============================================

print('✅ Pexels:', 'Configured')
print('✅ Pixabay:', 'Configured')
print('✅ Unsplash:', 'Configured')
print('✅ Coverr:', 'Configured')
print('✅ Freesound:', 'Configured')
print('✅ Freepik:', 'Configured')
print('\n🎉 All API keys ready! No manual input needed.')

In [ ]:
# Cell 3: Quick Test - Verify APIs Work
import requests
import os

print('Testing API connections...\n')

# Test Pexels
try:
    r = requests.get(
        'https://api.pexels.com/v1/search?query=nature&per_page=1',
        headers={'Authorization': os.environ['PEXELS_KEY']},
        timeout=10
    )
    print(f'Pexels: {"✅ OK" if r.status_code == 200 else "❌ Failed"} ({r.status_code})')
except Exception as e:
    print(f'Pexels: ❌ Error - {e}')

# Test Pixabay
try:
    r = requests.get(
        f'https://pixabay.com/api/videos/?key={os.environ["PIXABAY_KEY"]}&q=nature&per_page=3',
        timeout=10
    )
    print(f'Pixabay: {"✅ OK" if r.status_code == 200 else "❌ Failed"} ({r.status_code})')
except Exception as e:
    print(f'Pixabay: ❌ Error - {e}')

# Test Unsplash
try:
    r = requests.get(
        'https://api.unsplash.com/search/photos?query=nature&per_page=1',
        headers={'Authorization': f'Client-ID {os.environ["UNSPLASH_ACCESS_KEY"]}'},
        timeout=10
    )
    print(f'Unsplash: {"✅ OK" if r.status_code == 200 else "❌ Failed"} ({r.status_code})')
except Exception as e:
    print(f'Unsplash: ❌ Error - {e}')

# Test Freesound
try:
    r = requests.get(
        f'https://freesound.org/apiv2/search/text/?query=whoosh&token={os.environ["FREESOUND_KEY"]}',
        timeout=10
    )
    print(f'Freesound: {"✅ OK" if r.status_code == 200 else "❌ Failed"} ({r.status_code})')
except Exception as e:
    print(f'Freesound: ❌ Error - {e}')

print('\n✅ API tests complete!')

In [ ]:
# Cell 4: Launch Gradio UI (Easiest Method)
from uvg_core.gradio_ui import launch

# This will open a web UI where you can:
# - Paste your JSON script (schema v2.1)
# - Select resolution preset
# - Toggle captions
# - Generate video with one click

launch(share=True)

In [ ]:
# Cell 5: Alternative - Direct JSON Script Execution
# Use this cell if you prefer coding over UI

SCRIPT = '''
{
    "version": "2.1",
    "video_meta": {
        "title": "The Power of Focus",
        "narrative_style": "motivational",
        "orientation": "portrait",
        "resolution": {"width": 1080, "height": 1920},
        "include_captions": true,
        "thumbnail_enabled": true
    },
    "scenes": [
        {
            "scene_id": 1,
            "text": "In a world of endless distractions, focus is your superpower.",
            "duration_seconds": 5,
            "emotion": "dramatic",
            "voice_style": "motivational",
            "visual_descriptor": "person meditating at sunrise",
            "search_keywords": "meditation sunrise peace"
        },
        {
            "scene_id": 2,
            "text": "Every great achievement started with a single moment of clarity.",
            "duration_seconds": 5,
            "emotion": "calm",
            "voice_style": "cinematic",
            "visual_descriptor": "mountain peak at golden hour",
            "search_keywords": "mountain summit achievement"
        },
        {
            "scene_id": 3,
            "text": "Today, choose focus. Tomorrow, celebrate success.",
            "duration_seconds": 4,
            "emotion": "joyful",
            "voice_style": "warm",
            "visual_descriptor": "person celebrating victory",
            "search_keywords": "celebration success victory"
        }
    ]
}
'''

import json
from uvg_core.uvg_pipeline import run_from_json

script_data = json.loads(SCRIPT)
result = run_from_json(script_data)

print(f"{'✅ Success' if result.success else '❌ Failed'}: {result.output_path}")
if result.errors:
    for e in result.errors:
        print(f'Error: {e}')

In [ ]:
# Cell 6: View Generated Video
from IPython.display import Video, display
import glob
import os

videos = glob.glob('uvg_output/**/*.mp4', recursive=True)
if videos:
    latest = max(videos, key=os.path.getctime)
    print(f'Playing: {latest}')
    display(Video(latest, embed=True, width=400))
else:
    print('No video found. Run Cell 4 or 5 first!')

In [ ]:
# Cell 7: Download Output
from google.colab import files
import glob

output_files = glob.glob('uvg_output/**/*', recursive=True)
for f in output_files:
    if f.endswith(('.mp4', '.png', '.txt')):
        print(f'Downloading: {f}')
        files.download(f)

---

## 📋 JSON Schema v2.1 Reference

**Emotions:** `calm`, `exciting`, `dramatic`, `tense`, `joyful`, `sad`, `mysterious`, `neutral`

**Voice Styles:** `documentary`, `motivational`, `cinematic`, `warm`, `energetic`, `calm`, `dramatic`

```json
{
    "version": "2.1",
    "video_meta": {
        "title": "Video Title",
        "narrative_style": "cinematic | motivational | documentary | tiktok",
        "orientation": "portrait | landscape | square",
        "resolution": {"width": 1080, "height": 1920},
        "include_captions": true,
        "thumbnail_enabled": true,
        "deterministic_mode": false
    },
    "scenes": [
        {
            "scene_id": 1,
            "text": "Narration text",
            "duration_seconds": 5,
            "emotion": "calm",
            "voice_style": "documentary",
            "visual_descriptor": "description of ideal clip",
            "search_keywords": "pexels search terms"
        }
    ]
}
```

---

## 🎭 Emotion → Effect Mapping

| Emotion | VFX | Transition | Color | Voice |
|---------|-----|------------|-------|-------|
| calm | romantic | fade | warm | documentary |
| exciting | high_energy | whip_pan | neutral | energetic |
| dramatic | dramatic | cross_zoom | cold | cinematic |
| joyful | travel | light_leak | warm | warm |
| sad | vintage | dissolve | cold | calm |
| tense | noir | glitch | cold | dramatic |